In [1]:
import random
import pandas as pd
import numpy as np
import os
import cv2
import glob
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.transforms as transforms

import torchvision.models as models

from sklearn import metrics
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
import timm 

class model_MIL(nn.Module):
    def __init__(self):
        super().__init__()
        #self.backbone = models.efficientnet_b0(pretrained=True) # fixed
        self.backbone = timm.create_model('resnet50', pretrained=True , num_classes = 1)
        self.clf = nn.Sigmoid()
        
#         self.classifier = nn.Sequential(
#             nn.Linear(in_features=1000, out_features=1), # fixed
#             #nn.Linear(in_features=1024, out_features=1),
#             nn.Sigmoid(),
#         )
    def forward(self, x):
        x = self.backbone(x)
        x = self.clf(x)
        return x

In [4]:
import torch

from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor

m = model_MIL()
m.load_state_dict(torch.load('C:\\Users\\sanda\\Desktop\\dacon\\MIL_model_trained.pt'))

return_nodes = {
    'backbone.global_pool.flatten': 'img_feature',
}

fe = create_feature_extractor(m, return_nodes=return_nodes)
fe.to(device)
fe.eval()
    
print('모델 불러옴 (fe)')
# for key,item in res.items():
#     print(key)
#     print(item.shape)

모델 불러옴 (fe)


In [5]:
valid_T = transforms.Compose([
                        
                        #transforms.Grayscale(num_output_channels=1),
                        #transforms.Resize(image_resize), # (h, w) 순서
                        transforms.ToTensor(),
                        transforms.Normalize((0.5), (0.5))

                       ])

In [6]:
path = 'C:\\Users\\sanda\\Desktop\\dacon\\test_imgs_HnE_MIL224\\test\\False'

img_list = glob.glob(path + '\\*.'+ 'png')
#print(len(img_list))
img_name = []

for img_link in img_list:
    img_name.append(img_link.split('\\')[-1].split('.')[0].split('_')[:3][2])
    
img_name = list(set(img_name))    
print(len(img_name))

250


In [7]:
from PIL import Image

for name in tqdm(img_name):
    img_patch = glob.glob(path + '\\BC_01_'+ name+ '*.png')
    img_patch.sort(key = len)
    
    result_list = []
    
    for patch in img_patch:
        
        #print(patch)
        
        origin_img = Image.open(patch)

        img = valid_T(origin_img)

        with torch.no_grad():
        
            img = img.float().to(device)
                 
            img = img.unsqueeze(0)
        
            result = fe(img)
        
        #print(result['img_feature'].shape)
        
        result_list.append(result['img_feature'])
        
    #print(len(result_list))
    
    output = torch.stack(result_list, 0)
    #print(output.shape)
    output_np = output.cpu().numpy()
    np.save('C:\\Users\\sanda\\Desktop\\dacon\\img_features\\BC_01_'+name+'_img_feature', output_np)

  0%|          | 0/250 [00:00<?, ?it/s]

In [8]:
feature_list = glob.glob('C:\\Users\\sanda\\Desktop\\dacon\\img_features\\*')
len(feature_list)

1250

In [9]:
dim_list = []

for fe in feature_list:
    fe_np = np.load(fe)
    dim_list.append(fe_np.shape[0])

In [10]:
print(min(dim_list)) # 최소 패치
print(max(dim_list)) # 최대 패치
print(sum(dim_list) / len(dim_list)) # 평균 패치

1
242
68.9504


In [11]:
ti_list = glob.glob('C:\\Users\\sanda\\Desktop\\dacon\\train_imgs\\*')

In [12]:
a = []

for i in feature_list:
    name = i.split('\\')[-1]
    name = name[:10]
    a.append(name)

In [13]:
b = []

for t in ti_list:
    na = t.split('\\')[-1].split('.')[0]
    b.append(na)

In [14]:
print(len(a) , len(b))

1000 1000


In [15]:
print(set(b) - set(a)) # 이 이미지 HnE는 거의 비어있다.. 카테고리 1 True 다.

set()


In [148]:
img_f = np.load('C:\\Users\\sanda\\Desktop\\dacon\\img_features\\BC_01_0003_img_feature.npy')

In [149]:
img_f = torch.from_numpy(img_f)

In [150]:
#img_f = img_f.squeeze(1)

In [151]:
img_f.shape

torch.Size([152, 1, 2048])

In [152]:
a = img_f.permute(2,0,1)

In [153]:
a.shape

torch.Size([2048, 152, 1])

In [157]:
import torch.nn.functional as F
b = F.adaptive_max_pool2d(a, (1))

In [158]:
b.shape

torch.Size([2048, 1, 1])

In [162]:
b.squeeze(1).shape

torch.Size([2048, 1])